# Hallucination detection

In [1]:
!pip install -q transformers datasets
!pip install -q selfcheckgpt
!pip install -U -q deepeval

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 10.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.9/39.9 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.9 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 17.0.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 17.0.0 which is incompatible.
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 674.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 361.5/361.5 kB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install sentence-transformers metapub -q
!pip install transformers accelerate bitsandbytes -q
!pip install transformers trl py7zr optimum accelerate bitsandbytes -q

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.4/147.4 kB 11.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.4/177.4 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 16.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 40.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.8/245.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.9/67.9 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 18.9 MB/s eta 0:00:00
   ━━

In [ ]:
!pip install transformers datasets torch peft

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.4/296.4 kB 17.4 MB/s eta 0:00:00


In [4]:
# general packages
import torch
import random
import pandas as pd
from tqdm import tqdm

# Dataset

In [ ]:
from datasets import load_dataset

dataset = load_dataset("SajjadAyoubi/persian_qa")
df_train = pd.DataFrame([example for example in dataset["train"]])
df_train["selfcheckscore"] = None
df_train = df_train.drop(columns=['id'])


df_test = pd.DataFrame([example for example in dataset["validation"]])
df_test["selfcheckscore"] = None
df_test = df_test.drop(columns=['id'])

In [ ]:
df_train

,title,context,question,answers,selfcheckscore
0,شرکت فولاد مبارکه اصفهان,شرکت فولاد مبارکۀ اصفهان، بزرگ‌ترین واحد صنعتی...,شرکت فولاد مبارکه در کجا واقع شده است,"{'text': ['در شرق شهر مبارکه'], 'answer_start'...",None
1,شرکت فولاد مبارکه اصفهان,شرکت فولاد مبارکۀ اصفهان، بزرگ‌ترین واحد صنعتی...,فولاد مبارکه چند بار برنده جایزه شرکت دانشی را...,"{'text': ['۶'], 'answer_start': [263]}",None
2,شرکت فولاد مبارکه اصفهان,شرکت فولاد مبارکۀ اصفهان، بزرگ‌ترین واحد صنعتی...,شرکت فولاد مبارکه در سال ۱۳۹۱ چه جایزه ای برد؟,{'text': ['تندیس زرین جایزۀ ملی تعالی سازمانی'...,None
3,شرکت فولاد مبارکه اصفهان,شرکت فولاد مبارکۀ اصفهان، بزرگ‌ترین واحد صنعتی...,بزرگ ترین مجموعه تولید فولاد ایران چیست؟,"{'text': ['شرکت فولاد مبارکۀ'], 'answer_start'...",None
4,شرکت فولاد مبارکه اصفهان,شرکت فولاد مبارکۀ اصفهان، بزرگ‌ترین واحد صنعتی...,فولاد مبارکه در چه سالی احداث شد؟,"{'text': ['۱۳۷۱'], 'answer_start': [504]}",None
...,...,...,...,...,...
9003,شارب,شارِب به نوع ویژه‌ای از مو یا پرهای حسگر در بع...,همهٔ نخستیان به جز کدام موجود دارد شارب هستند؟,"{'text': ['انسان'], 'answer_start': [350]}",None
9004,شارب,شارِب به نوع ویژه‌ای از مو یا پرهای حسگر در بع...,شارب را می توان در کدام موجودات پیدا کرد؟,{'text': ['پستاندارانی چون سگ، گربه، خرس، و خو...,None
9005,شارب,شارِب به نوع ویژه‌ای از مو یا پرهای حسگر در بع...,عملکرد شارب در جانوران بی مهره چه گونه است؟,"{'text': [], 'answer_start': []}",None
9006,شارب,شارِب به نوع ویژه‌ای از مو یا پرهای حسگر در بع...,چرا شارب در بعضی هز جانوران خیلی مهم نیست؟,"{'text': [], 'answer_start': []}",None


In [ ]:
df_test

,title,context,question,answers,selfcheckscore
0,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,پایتخت اسپانیا کجاست؟,"{'text': ['مادرید', 'مادرید'], 'answer_start':...",None
1,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,بر چه اساسی رئال موفق ترین تیم در تاریخ فوتبال...,"{'text': ['فیفا', 'به انتخاب فیفا'], 'answer_s...",None
2,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,رئال مادرید چند بار در لیگ قهرمانان اروپا به ع...,"{'text': ['۱۳', '۱۳'], 'answer_start': [329, 3...",None
3,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,معنی واژه رئال به اسپانیایی چیست؟,"{'text': ['سلطنتی', 'سلطنتی'], 'answer_start':...",None
4,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,تیم رئال مادرید برای کجاست؟,"{'text': ['مادرید، پایتخت اسپانیا', 'اسپانیا']...",None
...,...,...,...,...,...
925,هندوانه,هندوانه یا تربوز (نام علمی: Citrullus lanatus)...,هندوانه در چه مناطقی می‌روید؟,{'text': ['هندوانه گیاهی گرمسیری یا نیمه گرمسی...,None
926,هندوانه,هندوانه یا تربوز (نام علمی: Citrullus lanatus)...,روش کاشت هندوانه در باغ را بنویسید.,{'text': ['دانه‌ها معمولاً در گلدان‌هایی دارای...,None
927,هندوانه,هندوانه یا تربوز (نام علمی: Citrullus lanatus)...,علت کاشت هندوانه مکعبی را بنویسید.,"{'text': [], 'answer_start': []}",None
928,هندوانه,هندوانه یا تربوز (نام علمی: Citrullus lanatus)...,گونه‌های هندوانه ایرانی را نام بیرید؟,"{'text': [], 'answer_start': []}",None


In [ ]:
df_train.iloc[5]['answers'], len(df_train), len(df_test)

({'text': ['در کمترین میزان خود، ۱٫۵٪ از دبی زاینده\u200cرود برابر سالانه ۲۳ میلیون متر مکعب در سال است'],
  'answer_start': [728]},
 9008,
 930)

# Decoder

In [ ]:
import gc
# del model
# del trainer
# del tokenizer
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

model_name_or_id = "MehdiHosseiniMoghadam/AVA-Mistral-7B-V2"
from transformers import AutoModelForCausalLM, AutoTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = AutoModelForCausalLM.from_pretrained(model_name_or_id, torch_dtype=torch.float16, device_map="auto", low_cpu_mem_usage=True, load_in_8bit=True)
tokenizer = AutoTokenizer.from_pretrained(model_name_or_id)

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.99,
    max_new_tokens=900,
    pad_token_id=tokenizer.eos_token_id
)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

def get_answer(question, context, model, generation_config):
    prompt = f'''

    با توجه به شرایط زیر به این سوال پاسخ دهید:
    مستقیما به پاسخ سوال در یک جمله پاسخ بده.
    ابتدا بازه جواب را در متن بنویس و سپس جواب را بنویس.

    {question}

    متن نوشته: {context}

    '''

    prompt = f"### Human:{prompt}\n### Assistant:"

    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(**inputs, generation_config=generation_config)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [ ]:
def create_prompt_df(df):
    for index, example in tqdm(df.iterrows()):
        question = example['question']
        context = example['context']
        answer = example['answers']
        prompt = f'''

        با توجه به شرایط زیر به این سوال پاسخ دهید:
        مستقیما به پاسخ سوال در یک جمله پاسخ بده.
        ابتدا بازه جواب را در متن بنویس و سپس جواب را بنویس.

        {question}

        متن نوشته: {context}

        '''


        prompt = f"### Human:{prompt}\n### Assistant:"
        df.loc[index, "prompt"] = prompt

        if len(answer['text']) == 0:
            answer_prompt = f'''
            جواب در متن وجود ندارد.
            '''
        else:
            answer_prompt = f'''
            بازه جواب: {answer['answer_start'][0]} تا {int(answer['answer_start'][0]) + len(answer['text'][0])}.

            متن جواب: {answer['text'][0]}

            '''

        df.loc[index, "answer_prompt"] = answer_prompt
    return df

df_train = create_prompt_df(df_train)
df_test = create_prompt_df(df_test)

9008it [00:06, 1493.87it/s]
930it [00:00, 2493.58it/s]


In [ ]:
print(df_train.iloc[5]['prompt'])
print(df_train.iloc[5]['answer_prompt'])

### Human:

        با توجه به شرایط زیر به این سوال پاسخ دهید:
        مستقیما به پاسخ سوال در یک جمله پاسخ بده.
        ابتدا بازه جواب را در متن بنویس و سپس جواب را بنویس.

        مصرف آب فولاد مبارکه چقدر است؟

        متن نوشته: شرکت فولاد مبارکۀ اصفهان، بزرگ‌ترین واحد صنعتی خصوصی در ایران و بزرگ‌ترین مجتمع تولید فولاد در کشور ایران است، که در شرق شهر مبارکه قرار دارد. فولاد مبارکه هم‌اکنون محرک بسیاری از صنایع بالادستی و پایین‌دستی است. فولاد مبارکه در ۱۱ دوره جایزۀ ملی تعالی سازمانی و ۶ دوره جایزۀ شرکت دانشی در کشور رتبۀ نخست را بدست آورده‌است و همچنین این شرکت در سال ۱۳۹۱ برای نخستین‌بار به عنوان تنها شرکت ایرانی با کسب امتیاز ۶۵۴ تندیس زرین جایزۀ ملی تعالی سازمانی را از آن خود کند. شرکت فولاد مبارکۀ اصفهان در ۲۳ دی ماه ۱۳۷۱ احداث شد و اکنون بزرگ‌ترین واحدهای صنعتی و بزرگترین مجتمع تولید فولاد در ایران است. این شرکت در زمینی به مساحت ۳۵ کیلومتر مربع در نزدیکی شهر مبارکه و در ۷۵ کیلومتری جنوب غربی شهر اصفهان واقع شده‌است. مصرف آب این کارخانه در کمترین میزان خود، ۱٫۵٪ از دبی زای

In [ ]:
tokenizer.pad_token = tokenizer.eos_token
# tokenizer.pad_token = "[PAD]"
# tokenizer.padding_side = "left"

In [ ]:
model.bfloat16()

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): MistralRotaryEmbedding()
        )
        (mlp): MistralMLP(
          (gate_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear8bitLt(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear8bitLt(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm()
        (post_attention_layernorm): MistralRMSNorm()
   

In [ ]:
print(df_test.iloc[5]['prompt'])
print('-------------------------------------')
print(df_test.iloc[5]['answer_prompt'])
print('-------------------------------------')
print(get_answer(df_test.iloc[5]['question'], df_test.iloc[5]['context'], model, generation_config))
print('---------------------------------')
inputs = tokenizer(df_test.iloc[5]['question'], return_tensors="pt").to(device)
outputs = model.generate(**inputs, generation_config=generation_config)
answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
print(answer)

### Human:

        با توجه به شرایط زیر به این سوال پاسخ دهید:
        مستقیما به پاسخ سوال در یک جمله پاسخ بده.
        ابتدا بازه جواب را در متن بنویس و سپس جواب را بنویس.

        لقب باشگاه رئال مادرید از کجا میاد؟

        متن نوشته: باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فوتبال است که در مادرید، پایتخت اسپانیا قرار دارد. رئال موفق‌ترین تیم تاریخ فوتبال اسپانیا و موفق‌ترین تیم تاریخ فوتبال اروپا و موفق‌ترین تیم فوتبال سدهٔ ۲۰ میلادی به انتخاب فیفا است. آن‌ها دارای رکورد ۳۴ بار قهرمانی در لالیگا، ۱۹ قهرمانی در کوپا دل ری (جام حذفی)، ۱۰ سوپرکاپ اسپانیا، ۱۳ قهرمانی در لیگ قهرمانان اروپا، ۲ جام یوفا، ۴ سوپرکاپ اروپا و ۴ قهرمانی در جام باشگاه‌های جهان هستند. رئال مادرید رکورددار قهرمانی در لیگ قهرمانان اروپا با ۱۳ قهرمانی و جام باشگاه های جهان با ۴ قهرمانی است. همچنین رئال مادرید برنده بهترین باشگاه قرن از سوی فیفا شده‌است. همچنین کلمهٔ رئال در زبان اسپانیایی، به معنی «سلطنتی» است، این لقب را شاه آلفونسو سیزدهم در سال ۱۹۲۰ بر این تیم نهاد. همچنین در همان سال، شکل یک تاج نیز بر رو

# evaluate by prompt engineering only

In [ ]:
limit = 20
for index, example in tqdm(df_test.iterrows()):
    print(index)
    if limit == index:
        break
    model_output_pe = get_answer(example['question'], example['context'], model, generation_config)
    df_test.loc[index, "model_output_pe"] = model_output_pe

0it [00:00, ?it/s]/home/amirhossein.hajimohammadrezaie/.conda/envs/diffmic/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


0


1it [00:09,  9.96s/it]

1


2it [03:48, 132.75s/it]

...

18


19it [30:29, 85.08s/it]

19


20it [30:47, 92.38s/it]

20


In [ ]:
df_test

,title,context,question,answers,selfcheckscore,prompt,answer_prompt,model_output_pe
0,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,پایتخت اسپانیا کجاست؟,"{'text': ['مادرید', 'مادرید'], 'answer_start':...",None,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 19 تا 25.\n\n ...,### Human:\n\n با توجه به شرایط زیر به این ...
1,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,بر چه اساسی رئال موفق ترین تیم در تاریخ فوتبال...,"{'text': ['فیفا', 'به انتخاب فیفا'], 'answer_s...",None,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 218 تا 222.\n\n ...,### Human:\n\n با توجه به شرایط زیر به این ...
2,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,رئال مادرید چند بار در لیگ قهرمانان اروپا به ع...,"{'text': ['۱۳', '۱۳'], 'answer_start': [329, 3...",None,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 329 تا 331.\n\n ...,### Human:\n\n با توجه به شرایط زیر به این ...
3,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,معنی واژه رئال به اسپانیایی چیست؟,"{'text': ['سلطنتی', 'سلطنتی'], 'answer_start':...",None,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 648 تا 654.\n\n ...,### Human:\n\n با توجه به شرایط زیر به این ...
4,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,تیم رئال مادرید برای کجاست؟,"{'text': ['مادرید، پایتخت اسپانیا', 'اسپانیا']...",None,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 61 تا 83.\n\n ...,### Human:\n\n با توجه به شرایط زیر به این ...
...,...,...,...,...,...,...,...,...
925,هندوانه,هندوانه یا تربوز (نام علمی: Citrullus lanatus)...,هندوانه در چه مناطقی می‌روید؟,{'text': ['هندوانه گیاهی گرمسیری یا نیمه گرمسی...,None,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 287 تا 392.\n\n ...,NaN
926,هندوانه,هندوانه یا تربوز (نام علمی: Citrullus lanatus)...,روش کاشت هندوانه در باغ را بنویسید.,{'text': ['دانه‌ها معمولاً در گلدان‌هایی دارای...,None,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 407 تا 539.\n\n ...,NaN
927,هندوانه,هندوانه یا تربوز (نام علمی: Citrullus lanatus)...,علت کاشت هندوانه مکعبی را بنویسید.,"{'text': [], 'answer_start': []}",None,### Human:\n\n با توجه به شرایط زیر به ...,\n جواب در متن وجود ندارد.\n ...,NaN
928,هندوانه,هندوانه یا تربوز (نام علمی: Citrullus lanatus)...,گونه‌های هندوانه ایرانی را نام بیرید؟,"{'text': [], 'answer_start': []}",None,### Human:\n\n با توجه به شرایط زیر به ...,\n جواب در متن وجود ندارد.\n ...,NaN


In [5]:
# df_test.to_pickle('df_test.pickle')
# df_test = pd.read_json('df_test.json')

In [13]:
from selfcheckgpt.modeling_selfcheck import SelfCheckNLI

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
selfcheck_nli = SelfCheckNLI(device=device) # set device to 'cuda' if GPU is available

def calc_self_gpt(df, column='model_output_pe'):
    for index, example in tqdm(df.iterrows()):
        model_output0 = example[column]

        main_answer = example["answer_prompt"]

        # pick after "Assistant"
        idx = model_output0.find("Assistant:")
        model_output0 = model_output0[idx + len("Assistant:"):]
        # print(model_output0)

        sent_scores_nli = selfcheck_nli.predict(
            sentences = [model_output0],   # model output
            sampled_passages = [main_answer], # main answer
        )
        df.loc[index, "selfcheckscore_pe"] = list(sent_scores_nli)[0] # Store the scores in the sent_scores_nli column
    return df

limit = 20
# df_train = calc_self_gpt(df_train)
df_test_sample = df_test.iloc[:limit]
df_test_sample = calc_self_gpt(df_test_sample)
df_test_sample

SelfCheck-NLI initialized to device cuda


0it [00:00, ?it/s]<ipython-input-13-9bde43fa102c>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.loc[index, "selfcheckscore_pe"] = list(sent_scores_nli)[0] # Store the scores in the sent_scores_nli column
20it [00:02,  8.95it/s]


,title,context,question,answers,selfcheckscore,prompt,answer_prompt,model_output_pe,selfcheckscore_pe
0,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,پایتخت اسپانیا کجاست؟,"{'text': ['مادرید', 'مادرید'], 'answer_start':...",NaN,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 19 تا 25.\n\n ...,### Human:\n\n با توجه به شرایط زیر به این ...,0.953394
1,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,بر چه اساسی رئال موفق ترین تیم در تاریخ فوتبال...,"{'text': ['فیفا', 'به انتخاب فیفا'], 'answer_s...",NaN,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 218 تا 222.\n\n ...,### Human:\n\n با توجه به شرایط زیر به این ...,0.553780
2,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,رئال مادرید چند بار در لیگ قهرمانان اروپا به ع...,"{'text': ['۱۳', '۱۳'], 'answer_start': [329, 3...",NaN,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 329 تا 331.\n\n ...,### Human:\n\n با توجه به شرایط زیر به این ...,0.961736
3,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,معنی واژه رئال به اسپانیایی چیست؟,"{'text': ['سلطنتی', 'سلطنتی'], 'answer_start':...",NaN,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 648 تا 654.\n\n ...,### Human:\n\n با توجه به شرایط زیر به این ...,0.187239
4,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,تیم رئال مادرید برای کجاست؟,"{'text': ['مادرید، پایتخت اسپانیا', 'اسپانیا']...",NaN,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 61 تا 83.\n\n ...,### Human:\n\n با توجه به شرایط زیر به این ...,0.834537
5,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,لقب باشگاه رئال مادرید از کجا میاد؟,{'text': ['شاه آلفونسو سیزدهم در سال ۱۹۲۰ بر ا...,NaN,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 672 تا 718.\n\n ...,### Human:\n\n با توجه به شرایط زیر به این ...,0.776784
6,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,دشمن اصلی باشگاه فوتبال رئال مادرید کدام تیم است؟,{'text': ['رقیب اصلی این تیم نیز، بارسلونا است...,NaN,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 857 تا 892.\n\n ...,### Human:\n\n با توجه به شرایط زیر به این ...,0.415937
7,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,بارسلونا چند بار قهرمان لیگ قهرمانان اروپا شده؟,"{'text': [], 'answer_start': []}",NaN,### Human:\n\n با توجه به شرایط زیر به ...,\n جواب در متن وجود ندارد.\n ...,### Human:\n\n با توجه به شرایط زیر به این ...,0.655468
8,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,باشگاه فوتبال رئال مادرید در چه سالی تاسیس شده؟,"{'text': [], 'answer_start': []}",NaN,### Human:\n\n با توجه به شرایط زیر به ...,\n جواب در متن وجود ندارد.\n ...,### Human:\n\n با توجه به شرایط زیر به این ...,0.467955
9,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,رکورددار بیشترین قهرمانی در لیگ قهرمانان آسیا ...,"{'text': [], 'answer_start': []}",NaN,### Human:\n\n با توجه به شرایط زیر به ...,\n جواب در متن وجود ندارد.\n ...,### Human:\n\n با توجه به شرایط زیر به این ...,0.750643


In [14]:
print(f'mean of selfcheck score for 20 samples: {df_test_sample["selfcheckscore_pe"].mean()}')

mean of selfcheck score for 20 samples: 0.6856685526669025


# Fine-tuning

In [ ]:
from datasets import Dataset

limit = 20

dataset_train = Dataset.from_pandas(df_train.iloc[:limit * 3])
dataset_test = Dataset.from_pandas(df_test.iloc[:limit])

def preprocess_function(examples):
    prompts = examples["prompt"]
    answers = examples["answer_prompt"]

    inputs = tokenizer(prompts, truncation=True, padding="max_length", max_length=512)
    labels = tokenizer(answers, truncation=True, padding="max_length", max_length=128)

    inputs["labels"] = labels["input_ids"]

    return inputs

encoded_dataset_train = dataset_train.map(preprocess_function, batched=True)
encoded_dataset_test = dataset_test.map(preprocess_function, batched=True)

Map:   0%|          | 0/60 [00:00<?, ? examples/s]

Map:   0%|          | 0/20 [00:00<?, ? examples/s]

In [ ]:
from peft import LoraConfig, get_peft_model
from peft import get_peft_model, LoraConfig, TaskType

# Define the adapter configuration
lora_config = LoraConfig(
    r=8,  # Number of attention heads in the adapter
    lora_alpha=32,  # Scaling factor
    lora_dropout=0.1,  # Dropout rate
    task_type=TaskType.CAUSAL_LM  # Task type for language models
)

# Apply the adapter configuration to the model
# model = get_peft_model(model, lora_config)
model.add_adapter(lora_config, adapter_name="adapter_1")


In [ ]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32002, 4096)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): lora.Linear8bitLt(
            (base_layer): Linear8bitLt(in_features=4096, out_features=4096, bias=False)
            (lora_dropout): ModuleDict(
              (adapter_1): Dropout(p=0.1, inplace=False)
            )
            (lora_A): ModuleDict(
              (adapter_1): Linear(in_features=4096, out_features=8, bias=False)
            )
            (lora_B): ModuleDict(
              (adapter_1): Linear(in_features=8, out_features=4096, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
            (lora_magnitude_vector): ModuleDict()
          )
          (k_proj): Linear8bitLt(in_features=4096, out_features=1024, bias=False)
          (v_proj): lora.Linear8bitLt(
            (base_layer): Linear8b

In [ ]:
print(get_answer(df_test.iloc[5]['question'], df_test.iloc[5]['context'], model, generation_config))

/home/amirhossein.hajimohammadrezaie/.conda/envs/diffmic/lib/python3.8/site-packages/bitsandbytes/autograd/_functions.py:316: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


### Human:

    با توجه به شرایط زیر به این سوال پاسخ دهید:
    مستقیما به پاسخ سوال در یک جمله پاسخ بده.
    ابتدا بازه جواب را در متن بنویس و سپس جواب را بنویس.

    لقب باشگاه رئال مادرید از کجا میاد؟

    متن نوشته: باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فوتبال است که در مادرید، پایتخت اسپانیا قرار دارد. رئال موفق‌ترین تیم تاریخ فوتبال اسپانیا و موفق‌ترین تیم تاریخ فوتبال اروپا و موفق‌ترین تیم فوتبال سدهٔ ۲۰ میلادی به انتخاب فیفا است. آن‌ها دارای رکورد ۳۴ بار قهرمانی در لالیگا، ۱۹ قهرمانی در کوپا دل ری (جام حذفی)، ۱۰ سوپرکاپ اسپانیا، ۱۳ قهرمانی در لیگ قهرمانان اروپا، ۲ جام یوفا، ۴ سوپرکاپ اروپا و ۴ قهرمانی در جام باشگاه‌های جهان هستند. رئال مادرید رکورددار قهرمانی در لیگ قهرمانان اروپا با ۱۳ قهرمانی و جام باشگاه های جهان با ۴ قهرمانی است. همچنین رئال مادرید برنده بهترین باشگاه قرن از سوی فیفا شده‌است. همچنین کلمهٔ رئال در زبان اسپانیایی، به معنی «سلطنتی» است، این لقب را شاه آلفونسو سیزدهم در سال ۱۹۲۰ بر این تیم نهاد. همچنین در همان سال، شکل یک تاج نیز بر روی آرم این باشگاه قرا

In [ ]:
from transformers import Seq2SeqTrainingArguments
from transformers import AutoModelForCausalLM, AutoTokenizer, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=10,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=encoded_dataset_train,
    eval_dataset=encoded_dataset_test,
    tokenizer=tokenizer
)

trainer.train()

# load model

In [ ]:
!unzip checkpoint-4500.zip -d checkpoint-4500

Archive:  checkpoint-4500.zip
  inflating: checkpoint-4500/checkpoint-4500/adapter_model.safetensors  
  inflating: checkpoint-4500/checkpoint-4500/added_tokens.json  
  inflating: checkpoint-4500/checkpoint-4500/rng_state.pth  
  inflating: checkpoint-4500/checkpoint-4500/tokenizer_config.json  
  inflating: checkpoint-4500/checkpoint-4500/special_tokens_map.json  
  inflating: checkpoint-4500/checkpoint-4500/optimizer.pt  
  inflating: checkpoint-4500/checkpoint-4500/scheduler.pt  
  inflating: checkpoint-4500/checkpoint-4500/tokenizer.json  
  inflating: checkpoint-4500/checkpoint-4500/generation_config.json  
  inflating: checkpoint-4500/checkpoint-4500/tokenizer.model  
  inflating: checkpoint-4500/checkpoint-4500/training_args.bin  
  inflating: checkpoint-4500/checkpoint-4500/adapter_config.json  
  inflating: checkpoint-4500/checkpoint-4500/trainer_state.json  


In [ ]:
import gc
# del model
gc.collect()
torch.cuda.empty_cache()

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, GenerationConfig

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


model = AutoModelForCausalLM.from_pretrained('checkpoint-4500/checkpoint-4500',  torch_dtype=torch.float16, device_map="auto", low_cpu_mem_usage=True, load_in_8bit=True)

tokenizer = AutoTokenizer.from_pretrained('checkpoint-4500/checkpoint-4500')

generation_config = GenerationConfig(
    do_sample=True,
    top_k=1,
    temperature=0.99,
    max_new_tokens=900,
    pad_token_id=tokenizer.eos_token_id
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


# eval

In [ ]:
df_test

,title,context,question,answers,selfcheckscore,prompt,answer_prompt
0,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,پایتخت اسپانیا کجاست؟,"{'text': ['مادرید', 'مادرید'], 'answer_start':...",None,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 19 تا 25.\n\n ...
1,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,بر چه اساسی رئال موفق ترین تیم در تاریخ فوتبال...,"{'text': ['فیفا', 'به انتخاب فیفا'], 'answer_s...",None,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 218 تا 222.\n\n ...
2,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,رئال مادرید چند بار در لیگ قهرمانان اروپا به ع...,"{'text': ['۱۳', '۱۳'], 'answer_start': [329, 3...",None,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 329 تا 331.\n\n ...
3,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,معنی واژه رئال به اسپانیایی چیست؟,"{'text': ['سلطنتی', 'سلطنتی'], 'answer_start':...",None,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 648 تا 654.\n\n ...
4,باشگاه فوتبال رئال مادرید,باشگاه فوتبال رئال مادرید یک باشگاه حرفه‌ای فو...,تیم رئال مادرید برای کجاست؟,"{'text': ['مادرید، پایتخت اسپانیا', 'اسپانیا']...",None,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 61 تا 83.\n\n ...
...,...,...,...,...,...,...,...
925,هندوانه,هندوانه یا تربوز (نام علمی: Citrullus lanatus)...,هندوانه در چه مناطقی می‌روید؟,{'text': ['هندوانه گیاهی گرمسیری یا نیمه گرمسی...,None,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 287 تا 392.\n\n ...
926,هندوانه,هندوانه یا تربوز (نام علمی: Citrullus lanatus)...,روش کاشت هندوانه در باغ را بنویسید.,{'text': ['دانه‌ها معمولاً در گلدان‌هایی دارای...,None,### Human:\n\n با توجه به شرایط زیر به ...,\n بازه جواب: 407 تا 539.\n\n ...
927,هندوانه,هندوانه یا تربوز (نام علمی: Citrullus lanatus)...,علت کاشت هندوانه مکعبی را بنویسید.,"{'text': [], 'answer_start': []}",None,### Human:\n\n با توجه به شرایط زیر به ...,\n جواب در متن وجود ندارد.\n ...
928,هندوانه,هندوانه یا تربوز (نام علمی: Citrullus lanatus)...,گونه‌های هندوانه ایرانی را نام بیرید؟,"{'text': [], 'answer_start': []}",None,### Human:\n\n با توجه به شرایط زیر به ...,\n جواب در متن وجود ندارد.\n ...


In [ ]:
for index, example in tqdm(df_test.iterrows()):
    prompt = example['prompt']
    answer = example['answer_prompt']
    model_output0 = get_answer(example['question'], example['context'], model, generation_config)
    df_test.loc[index, "model_output0"] = model_output0

In [ ]:
from selfcheckgpt.modeling_selfcheck import SelfCheckNLI

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
selfcheck_nli = SelfCheckNLI(device=device) # set device to 'cuda' if GPU is available

tokenizer_config.json:   0%|          | 0.00/400 [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/883 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/1.74G [00:00<?, ?B/s]

SelfCheck-NLI initialized to device cuda


In [ ]:
def calc_self_gpt(df):
    for index, example in tqdm(df.iterrows()):
        model_output0 = example["model_output0"]

        if example["answers"]['text'] == []:
            main_answer = "پاسخ ندارد"
        else:
            main_answer = example["answers"]['text'][0]

        sent_scores_nli = selfcheck_nli.predict(
            sentences = [model_output0],   # model output
            sampled_passages = [main_answer], # main answer
        )
        df.loc[index, "selfcheckscore"] = list(sent_scores_nli)[0] # Store the scores in the sent_scores_nli column
    return df

# df_train = calc_self_gpt(df_train)
df_test = calc_self_gpt(df_test)

9008it [07:17, 20.57it/s]
930it [00:43, 21.29it/s]


In [ ]:
df_test